We use this script to process the Gesture dataset and get the input files for the GRU-D models.

In [1]:
import os 
import numpy as np
import pandas as pd
from python_linq import From

In [2]:
seed = 42
np.random.seed(seed=seed)

In [15]:
gesture_root_folder = os.path.join('gesture_datas/')
grud_working_folder = '.' # # This should be the [WD] (working directory) of the GRU-D project.
grud_gesture_data_folder = os.path.join(grud_working_folder, 'data', 'gesture')
if not os.path.exists(grud_gesture_data_folder):
    os.makedirs(grud_gesture_data_folder)
print('The data files will be converted and saved here: ', grud_gesture_data_folder)

The data files will be converted and saved here:  ./data/gesture


In [19]:
def Load_data(path):
    all_files = os.listdir(path)
    files_to_load = From(all_files).where(lambda file: file[-3:]=='csv').where(lambda file: file[-5]=='w').toList()
    df_Big = pd.read_csv(path +files_to_load[0], sep=',')
    for file in files_to_load:
        df_file = pd.read_csv(path + file, sep=',')
        df_Big = df_Big.append(df_file)
    df_Big = df_Big.loc[df_Big['phase']!='Preparação']
    return df_Big
dataframe = Load_data(gesture_root_folder)

In [20]:
def Truncate(df):
    k = 0
    nb_rows = df.shape[0]
    df_to_fill = pd.DataFrame(columns=df.columns.values.tolist())
    while k < nb_rows:    
        row_considered = df.iloc[[k]]
        phase_considered = row_considered['phase'].values.tolist()[0]
        phase_tmp = phase_considered
        #print(phase_considered)
        #print(phase_tmp)
        i = 0
        while phase_tmp == phase_considered and i<30:
            if k + i + 1 < nb_rows:
                # Verifying if there is a next value
                i += 1
                phase_tmp = df.iloc[[i+k]]['phase'].values.tolist()[0]
            else:
                # End of while
                break
        if i > 7:
            # Add the row considered            
            df_to_fill = df_to_fill.append(row_considered)            
        if k + i + 1 < nb_rows:
            k += i
        else:
            # End of the dataframe
            break
    df_to_fill = df_to_fill.reset_index(drop=True)
    return df_to_fill
df_truncated = Truncate(dataframe)

In [21]:
def Creat_missing_values(df, nb_rows_to_del=1):
    nb_rows = df.shape[0]
    nb_columns = df.shape[1]
    index_still = np.arange(nb_rows)
    df_copy = df.copy()
    if nb_rows_to_del < nb_rows:
        while nb_rows-len(index_still)<nb_rows_to_del:
            index_del = np.random.randint(len(index_still)-1)
            timestamp_saved = df.iloc[index_del].__getitem__('timestamp')
            df_copy.iloc[index_del] = [np.nan for i in range(nb_columns)]
            df_copy.loc[index_del, 'timestamp'] = timestamp_saved
            index_still = np.delete(index_still,index_del)
    return df_copy
df_truncated_missing_data = Creat_missing_values(df_truncated)

In [26]:
nb_rows = df_truncated_missing_data.shape[0]
nb_columns = df_truncated_missing_data.shape[1]
mask = np.zeros(nb_rows*nb_columns).reshape((nb_rows,nb_columns))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
keys = ['lhx', 'lhy', 'lhz', 'rhx', 'rhy', 'rhz', 'hx', 'hy', 'hz', 'sx', 'sy', 'sz', 'lwx', 'lwy', 'lwz', 'rwx', 'rwy', 'rwz', 'timestamp', 'phase']
input_keys = ['lhx', 'lhy', 'lhz', 'rhx', 'rhy', 'rhz', 'hx', 'hy', 'hz', 'sx', 'sy', 'sz', 'lwx', 'lwy', 'lwz', 'rwx', 'rwy', 'rwz']
timestamp_key = 'timestamp'

In [22]:
def Creat_mask(df):
    

0      2730074
1      2731073
2      2731641
3      2732639
4      2733471
        ...   
513    5429382
514    5430306
515    5430571
516    5431038
517    5432169
Name: timestamp, Length: 518, dtype: int64

In [ ]:
timestamp = df_truncated_missing_data.timestamp
processed_data = {
    'input': [],
    'masking': []
}
n = len(timestamp)

In [ ]:
# Convert *raw.csv to data.npz
raw_data = load(gesture_root_folder) #TODO: Creat load() that load all csv data(clean data and make appropriate format)
timestamp = raw_data['T_t']
processed_data = {
    'input' : raw_data['X_t'],
    'masking' : raw_data['X_t_mask']
}
n = len(timestamp)

In [ ]:
def load(path):
    raw_data = {
        'T_t' : [],
        'X_t' : [],
        'x_t_mask' : []
    }
    
    return raw_data